# COMPAS MLP FTU - mk2

## Load Dependencies <a id="libraries"></a>

In [1]:
import pandas as pd
import numpy as np
import timeit
import os
import math

In [2]:
import torch
import torch.nn as nn
import torchmetrics
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [4]:
import warnings
warnings.filterwarnings('ignore')

### Notebook options and parameters

In [5]:
pd.options.display.max_columns = 250
pd.options.display.max_rows = 250

In [6]:
# random seed for consistency in pytorch and numpy
SEED = 23

torch.manual_seed(SEED)
np.random.seed(SEED)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Hyperparameter options

In [8]:
# activation functions
elu = nn.ELU()
leaky = nn.LeakyReLU()
relu = nn.ReLU()
tanh = nn.Tanh()
sigmoid = nn.Sigmoid()

# loss functions
mse = nn.MSELoss()
bce = nn.BCELoss()
kld = nn.KLDivLoss()

# optimizers
adam = torch.optim.Adam
sgd = torch.optim.SGD

## Custom Functions <a id="functions"></a>

#### Create dataset for DataLoader

In [9]:
class dataset(Dataset):
    def __init__(self, data, target):
        self.X = torch.tensor(data, dtype=torch.float32, device=device)
        self.y = torch.tensor(target, dtype=torch.float32, device=device)
        
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def __len__(self):
        return len(self.X)

#### Training function

In [10]:
def train(data, model, criterion, metric, optimizer):
    size = len(data.dataset)
    batches = len(data)
    running_loss = []
    acc = 0.0
    correct = 0
    predictions = []
    actuals = []
    loss = 0.0
    acc = 0.0
    current = 0
    
    for batch, (X, y) in enumerate(data,1):
        
        # forward step
        optimizer.zero_grad() # zero out the gradients
        y_hat = model(X)  #forward pass
        
        y = y.unsqueeze(-1) # reduce dimensions of tensor from [256, 1] to [256]
        
        loss = criterion(y_hat, y)  # calculate loss
       
        #  backprop step
        loss.backward()       # backward pass through model; computes gradients
        optimizer.step()      # update weights
        
        # metrics
        y = y.int()
        acc = metric(y_hat, y)

        y_hat = y_hat.cpu().detach().numpy()
        y_hat = y_hat.reshape(-1)
        
        y = y.cpu().detach().numpy()
        y = y.reshape(-1)
        
        predictions.append(y_hat)
        actuals.append(y)
            
        running_loss.append(loss.item())
    
        ## every 50 batches, store the results
        #if batch % 50 == 0:
        #    current = batch * len(X)
        
    print(f"\tTraining\tAccuracy: {acc:1.10f}\tLoss: {loss:1.10f}")
    
    return predictions, running_loss

#### Testing function

In [11]:
def test(data, model, criterion, metric):
    size = len(data.dataset)
    batches = len(data)
    tst_loss = 0
    correct = 0
    predictions = []
    actuals = []
    
    with torch.no_grad():
        for X, y in data:
            y_hat = model(X)
            y = y.unsqueeze(-1)
            tst_loss += criterion(y_hat, y).item()
            
            y = y.int()
            acc = metric(y_hat, y)
            #acc = acc.cpu().detach().numpy()
            
            y_hat = y_hat.cpu().detach().numpy()
            y_hat = y_hat.reshape(-1)
        
            y = y.cpu().detach().numpy()
            y = y.reshape(-1)
        
            predictions.append(y_hat)
            actuals.append(y)
            
    tst_loss /= batches
    correct /= size
    
    print(f"\tTest error\tAccuracy: {acc:1.10f}\tLoss: {tst_loss:1.10f}")
    return predictions, tst_loss, acc

#### Evaluation - train and test

In [12]:
def evaluate(training_data, test_data, model, loss_function, metric, optimizer, epochs, early_stop=True):
    training_results = []
    training_actuals = []
    training_losses = []

    testing_results = []
    testing_actuals = []

    start = timeit.default_timer()
    
    # early stop criteria
    
    default_patience = 3
    patience = default_patience
    running_loss = []
    running_acc = []

    for epoch in range(1,epochs+1):
        print (f"Epoch {epoch} / {epochs}")

        trng_result, trng_loss = train(training_data, model, loss_function, metric, optimizer)
        tst_result, tst_loss, acc = test(test_data, model, loss_function, metric)
        
        training_results.append(trng_result)
        training_losses.append(trng_loss)
        testing_results.append(tst_result)
        running_loss.append(tst_loss)
        running_acc.append(acc)
        
        # check for early stop
        if epoch > 3:            
            if early_stop and ((tst_loss >= running_loss[-2]) or math.isclose(acc,running_acc[-2], abs_tol=1e-8)):
                patience = patience - 1
            elif early_stop and (tst_loss < running_loss[-2]):
                patience = default_patience

            if early_stop and (patience <= 0):
                print("Early stop - ", end="")
                break

    
    print("Finished")
    print("* " * 30)
    stop = timeit.default_timer()
    print(f"Execution time (in seconds): {stop - start}")

    return training_results, training_losses, testing_results

#### Multilayer Perception (MLP)

In [13]:
class MLP(nn.Module):
    
    def __init__(self, in_size, hidden1=relu, hidden2=relu, hidden3=relu, out_layer=sigmoid):
        super(MLP, self).__init__()
        
        # construct model dependent on size of inputs
        out1_2 = 2*in_size // 3
        out2_3 = 2*out1_2 // 3
        out3_4 = 2*out2_3 // 3

        self.linear_stack = nn.Sequential(
            nn.Linear(in_size, out1_2),
            hidden1,
            nn.Linear(out1_2, out2_3),
            hidden2,
            nn.Linear(out2_3, out3_4),
            hidden3,
            nn.Linear(out3_4, 1),
            out_layer
        )
        
    def forward(self, x):
        out = self.linear_stack(x)

        return out

#### Flatten function

In [14]:
def flatten(thing):
    '''
    thing: a list
    
    flatten receives a multi-level list and flatten it down 2 layers
    '''
    #thing = [element for sublist in thing for element in sublist]
    return [element for sublist in thing for element in sublist]

#### Prepare data for eval and analysis

In [15]:
def prep(name, training_results, testing_results):
    trng_preds = [[] for _ in range(len(training_results))]

    tst_preds = [[] for _ in range(len(testing_results))]
    
    for i in range(len(training_results)):
        trng_preds[i] = flatten(training_results[i])

        tst_preds[i] = flatten(testing_results[i])

    return trng_preds, tst_preds

#### Send results to tensorboard

In [16]:
def to_tensorboard(tailend, directory, dataset, train_preds, test_preds, y_train, y_test):
    acc_trng_relu = []
    acc_tst_relu = []
    loss_relu = []

    writer = SummaryWriter(log_dir=directory)

    for i in range(len(train_preds)):
        y_hat = np.array(np.round(train_preds[i]))
        y_hat_tst = np.array(np.round(test_preds[i]))
        trng_acc = (y_hat == y_train).sum()/len(train_preds[i])
        test_acc = (y_hat_tst == y_test).sum()/len(test_preds[i])

        avg_loss = np.average(training_losses[i])

        acc_trng_relu.append(trng_acc)
        acc_tst_relu.append(test_acc)
        loss_relu.append(avg_loss)
        writer.add_scalar(f"Training Accuracy {tailend}", trng_acc, i)
        writer.add_scalar(f"Test Accuracy {tailend}", test_acc, i)
        writer.add_scalar(f"Training Loss {tailend}", avg_loss, i)
        writer.add_pr_curve(f"Precision-Recall Curve {tailend}", y_test, y_hat_tst, i)

    t, _ = next(iter(dataset))
    writer.add_graph(model, t)

    writer.close()

#### Save model

In [17]:
def custom_save(name, data, kind=1):
    '''
    name : string
        designated filename
    data : data or pytorch model
        the data to save
    kind : int
        sentinel value - 1 if pytorch model, 0 otherwise
    
    custom_save stores the data passed into the function into a file with the provided name
    '''
    
    if kind == 1:
        ex = ".pth"
    else:
        ex = ".parquet"
    
    sentinel = True
    i = 1

    while sentinel:
        dirlist = os.listdir()

        if name not in dirlist:
            if kind == 1:
                torch.save(data, name)
            else:
                data.to_parquet(name)
            print(f"{name} has been saved.")                
            sentinel = False
        if name in dirlist:
            print(f"{name} already exists.", end=" ")
            temp, ext = name.split(ex)
            if "_v" in temp:
                temp, _ = temp.split("_v")
            name = f"{temp}_v{i}{ex}"
            i = i + 1
            print(f"Changing file name to: {name}")

# Import Data<a id="data"></a>
##### COMPAS dataset

#### Uncomment next when using google colab

In [18]:
#data = pd.read_parquet("https://github.com/ollin23/bias/blob/main/compas_mk1_v1.parquet?raw=true")

#### Use the following cell if the data is stored locally

In [19]:
data = pd.read_parquet("compas_mk1_v1.parquet")

#### Peak the data

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6907 entries, 0 to 7213
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      6907 non-null   int64  
 1   juv_fel_count            6907 non-null   int64  
 2   decile_score             6907 non-null   int64  
 3   juv_misd_count           6907 non-null   int64  
 4   juv_other_count          6907 non-null   int64  
 5   priors_count             6907 non-null   int64  
 6   days_b_screening_arrest  6907 non-null   float64
 7   c_days_from_compas       6907 non-null   float64
 8   is_recid                 6907 non-null   int64  
 9   is_violent_recid         6907 non-null   int64  
 10  decile_score.1           6907 non-null   int64  
 11  v_decile_score           6907 non-null   int64  
 12  priors_count.1           6907 non-null   int64  
 13  start                    6907 non-null   int64  
 14  end                     

In [21]:
data.shape

(6907, 34)

In [22]:
data.head()

,age,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,is_recid,is_violent_recid,decile_score.1,v_decile_score,priors_count.1,start,end,sex,age_cat_25to45,age_cat_over45,age_cat_under25,race_black,race_asian,race_white,race_hispanic,race_native,race_Other,c_charge_degree,score_text_High,score_text_Low,score_text_Medium,v_score_text_High,v_score_text_Low,v_score_text_Medium,event,target
0,69,0,1,0,0,0,-1.0,1.0,0,0,1,1,0,0,327,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0
1,34,0,3,0,0,0,-1.0,1.0,1,1,3,1,0,9,159,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,1,1
2,24,0,4,0,1,4,-1.0,1.0,1,0,4,3,4,0,63,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,1,0,0,1
5,44,0,1,0,0,0,0.0,0.0,0,0,1,1,0,1,853,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0
6,41,0,6,0,0,14,-1.0,1.0,1,0,6,2,14,5,40,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,1,1


In [23]:
list(data.columns)

['age',
 'juv_fel_count',
 'decile_score',
 'juv_misd_count',
 'juv_other_count',
 'priors_count',
 'days_b_screening_arrest',
 'c_days_from_compas',
 'is_recid',
 'is_violent_recid',
 'decile_score.1',
 'v_decile_score',
 'priors_count.1',
 'start',
 'end',
 'sex',
 'age_cat_25to45',
 'age_cat_over45',
 'age_cat_under25',
 'race_black',
 'race_asian',
 'race_white',
 'race_hispanic',
 'race_native',
 'race_Other',
 'c_charge_degree',
 'score_text_High',
 'score_text_Low',
 'score_text_Medium',
 'v_score_text_High',
 'v_score_text_Low',
 'v_score_text_Medium',
 'event',
 'target']

### Process Data for Neural Net Usage <a id="convert"></a>

#### Split features into X and y

In [24]:
X = data.drop("target", axis=1).values.astype(np.float32)
y = data.target.values.astype(np.float32)

#### Standardize data

In [25]:
scaler = StandardScaler()
scaledX = scaler.fit_transform(X)

#### Split into training and testing sets

In [26]:
X_train, X_test, y_train, y_test = train_test_split(scaledX, y, test_size=0.3,
                                                        random_state=SEED, stratify=y)

In [27]:
X_train.shape

(4834, 33)

In [28]:
X_test.shape

(2073, 33)

In [29]:
col = list(data.columns).index("race_black")
col

19

In [30]:
data.columns[19:25]

Index(['race_black', 'race_asian', 'race_white', 'race_hispanic',
       'race_native', 'race_Other'],
      dtype='object')

black = X_test[:, 19]
asian = X_test[:, 20]
white = X_test[:, 21]
hispanic = X_test[:, 22]
native = X_test[:, 23]
other = X_test[:,24]

In [31]:
temp = np.round(scaler.inverse_transform(X_test)).astype(int)

In [32]:
X_train = np.delete(X_train, [19,20,21,22,23,24], axis=1)
X_test = np.delete(X_test, [19,20,21,22,23,24], axis=1)

#### Make datasets

In [33]:
scaled_train = dataset(X_train, y_train)
scaled_test = dataset(X_test, y_test)

##### Construct test results dataframe for persistent storage

In [34]:
features = list(data.drop("target", axis=1).columns)

df = pd.DataFrame(temp, columns=features)
black = df.race_black.values
white = df.race_white.values
hispanic = df.race_hispanic.values
native = df.race_native.values
other = df.race_Other.values

# drop old 
df.drop(["race_black", "race_white", "race_hispanic", "race_native", \
         "race_Other"], axis=1, inplace=True)

df["black"] = black
df["white"] = white
df["hispanic"] = hispanic
df["native"] = native
df["other"] = other
df["target"] = y_test.astype(int)

In [35]:
df.head()

,age,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,is_recid,is_violent_recid,decile_score.1,v_decile_score,priors_count.1,start,end,sex,age_cat_25to45,age_cat_over45,age_cat_under25,race_asian,c_charge_degree,score_text_High,score_text_Low,score_text_Medium,v_score_text_High,v_score_text_Low,v_score_text_Medium,event,black,white,hispanic,native,other,target
0,25,0,5,0,1,4,-1,1,1,0,5,4,4,32,304,0,1,0,0,0,1,0,0,1,0,1,0,1,1,0,0,0,0,1
1,35,0,4,0,0,3,-1,0,1,0,4,4,3,9,462,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1
2,32,0,4,0,0,1,0,0,0,0,4,3,1,0,1171,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0
3,20,0,10,0,0,0,-1,1,1,1,10,10,0,3,20,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,0,1
4,50,0,8,0,2,24,0,1,1,1,8,8,24,32,87,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1


##### Note
For this dataset: Female == 1, Male == 0

# Experiment Phase 3 - COMPAS Data <a id="experiments"></a>

## Trial 3.1 - ReLU (COMPAS) <a id="relu"></a>

### Preliminaries

In [36]:
suffix = "mk2_FTU"
directory = f"COMPAS_{suffix}"
prefix = "compas"

#### Hyperparameters <a id="hyperparameters"></a>

In [37]:
alpha = 1e-5 # 0.00001
epochs = 150
batches = 100
hidden1 = relu
hidden2 = relu
hidden3 = relu
out_layer = sigmoid

#### DataLoader

In [38]:
# Helper function to feed data into pytorch neural network, only needs to execute once
training_s = DataLoader(scaled_train, batch_size=batches)
test_s = DataLoader(scaled_test, batch_size=batches)

#### Construct Model

In [39]:
# instantiate model
model = MLP(X_train.shape[1], hidden1, hidden2, hidden3, out_layer).to(device)

# select optimizing function
optimizer = adam(model.parameters(), lr=alpha)

# select loss function
loss_function = mse

# accuracy metric
metric = torchmetrics.functional.accuracy

# display model
print(model.parameters)

<bound method Module.parameters of MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=27, out_features=18, bias=True)
    (1): ReLU()
    (2): Linear(in_features=18, out_features=12, bias=True)
    (3): ReLU()
    (4): Linear(in_features=12, out_features=8, bias=True)
    (5): ReLU()
    (6): Linear(in_features=8, out_features=1, bias=True)
    (7): Sigmoid()
  )
)>


#### Train and test

In [40]:
training_results = []
training_losses = []
testing_results = []

training_results, training_losses, testing_results = evaluate(training_s,
                                                            test_s,
                                                            model,
                                                            loss_function,
                                                            metric,
                                                            optimizer,
                                                            epochs,
                                                            early_stop=False)

Epoch 1 / 150
	Training	Accuracy: 0.5588235259	Loss: 0.2482961714
	Test error	Accuracy: 0.6027397513	Loss: 0.2482213264
Epoch 2 / 150
	Training	Accuracy: 0.5588235259	Loss: 0.2479248643
	Test error	Accuracy: 0.6027397513	Loss: 0.2478888127
Epoch 3 / 150
	Training	Accuracy: 0.5588235259	Loss: 0.2475468665
	Test error	Accuracy: 0.6027397513	Loss: 0.2475545201
Epoch 4 / 150
	Training	Accuracy: 0.5588235259	Loss: 0.2471761107
	Test error	Accuracy: 0.6027397513	Loss: 0.2472212733
Epoch 5 / 150
	Training	Accuracy: 0.5588235259	Loss: 0.2467991859
	Test error	Accuracy: 0.6027397513	Loss: 0.2468845489
Epoch 6 / 150
	Training	Accuracy: 0.5588235259	Loss: 0.2464226633
	Test error	Accuracy: 0.6027397513	Loss: 0.2465480445
Epoch 7 / 150
	Training	Accuracy: 0.5588235259	Loss: 0.2460504472
	Test error	Accuracy: 0.6027397513	Loss: 0.2462079475
Epoch 8 / 150
	Training	Accuracy: 0.5588235259	Loss: 0.2456572801
	Test error	Accuracy: 0.6027397513	Loss: 0.2458682600
Epoch 9 / 150
	Training	Accuracy: 0.5588

	Training	Accuracy: 0.7647058964	Loss: 0.2107095271
	Test error	Accuracy: 0.8082191944	Loss: 0.2151189418
Epoch 71 / 150
	Training	Accuracy: 0.7647058964	Loss: 0.2099732757
	Test error	Accuracy: 0.8082191944	Loss: 0.2144449524
Epoch 72 / 150
	Training	Accuracy: 0.7647058964	Loss: 0.2092434317
	Test error	Accuracy: 0.8082191944	Loss: 0.2137653750
Epoch 73 / 150
	Training	Accuracy: 0.7941176295	Loss: 0.2085102648
	Test error	Accuracy: 0.8082191944	Loss: 0.2130737794
Epoch 74 / 150
	Training	Accuracy: 0.7941176295	Loss: 0.2077711076
	Test error	Accuracy: 0.8219178319	Loss: 0.2123877754
Epoch 75 / 150
	Training	Accuracy: 0.7941176295	Loss: 0.2070229799
	Test error	Accuracy: 0.8219178319	Loss: 0.2116830271
Epoch 76 / 150
	Training	Accuracy: 0.7941176295	Loss: 0.2062654942
	Test error	Accuracy: 0.8219178319	Loss: 0.2109761650
Epoch 77 / 150
	Training	Accuracy: 0.7647058964	Loss: 0.2054741383
	Test error	Accuracy: 0.8219178319	Loss: 0.2102476408
Epoch 78 / 150
	Training	Accuracy: 0.7941176295

	Training	Accuracy: 0.9411764741	Loss: 0.1511273980
	Test error	Accuracy: 0.9452054501	Loss: 0.1590668822
Epoch 139 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1502156854
	Test error	Accuracy: 0.9452054501	Loss: 0.1581822116
Epoch 140 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1492963135
	Test error	Accuracy: 0.9452054501	Loss: 0.1572961715
Epoch 141 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1483827531
	Test error	Accuracy: 0.9452054501	Loss: 0.1564185570
Epoch 142 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1474650055
	Test error	Accuracy: 0.9452054501	Loss: 0.1555178173
Epoch 143 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1465667039
	Test error	Accuracy: 0.9452054501	Loss: 0.1546363412
Epoch 144 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1456639320
	Test error	Accuracy: 0.9452054501	Loss: 0.1537606298
Epoch 145 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1447634399
	Test error	Accuracy: 0.9452054501	Loss: 0.1528802464
Epoch 146 / 150
	Training	Accuracy: 0.94

### Prepare results for analysis

In [41]:
tailend = f"ReLU ({prefix})"

train_preds, test_preds = prep(tailend, training_results, testing_results)

#### Save ReLU Metrics to Tensorboard <a id="relu-metrics-for-tensorboard"></a>

In [42]:
to_tensorboard(tailend, directory, training_s, train_preds, test_preds, y_train, y_test)

#### Add ReLU predictions to persistent storage

In [43]:
df["pred_relu"] = np.array(np.round(test_preds[-1])).astype(int)

name = f"{prefix}_relu_{suffix}.pth"
custom_save(name, model)
#torch.save(model, f"{prefix}_relu_scaled.pth")

compas_relu_mk2_FTU.pth has been saved.


In [44]:
df.head()

,age,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,is_recid,is_violent_recid,decile_score.1,v_decile_score,priors_count.1,start,end,sex,age_cat_25to45,age_cat_over45,age_cat_under25,race_asian,c_charge_degree,score_text_High,score_text_Low,score_text_Medium,v_score_text_High,v_score_text_Low,v_score_text_Medium,event,black,white,hispanic,native,other,target,pred_relu
0,25,0,5,0,1,4,-1,1,1,0,5,4,4,32,304,0,1,0,0,0,1,0,0,1,0,1,0,1,1,0,0,0,0,1,1
1,35,0,4,0,0,3,-1,0,1,0,4,4,3,9,462,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,1
2,32,0,4,0,0,1,0,0,0,0,4,3,1,0,1171,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0
3,20,0,10,0,0,0,-1,1,1,1,10,10,0,3,20,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,0,1,1
4,50,0,8,0,2,24,0,1,1,1,8,8,24,32,87,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1


## Trial 3.2 - Tanh (COMPAS) <a id="tanh"></a>

#### Hyperparameters

In [45]:
#alpha = 1e-5 # 0.00001
#epochs = 50
#batches = 256
hidden1 = tanh
hidden2 = tanh
hidden3 = tanh
out_layer = sigmoid
print(f"learning rate: {alpha}")
print(f"epochs: {epochs}")
print(f"batches: {batches}")

learning rate: 1e-05
epochs: 150
batches: 100


#### Construct model

In [46]:
# instantiate model
model = MLP(X_train.shape[1], hidden1, hidden2, hidden3, out_layer).to(device)

# select optimizing function
optimizer = adam(model.parameters(), lr=alpha)

# select loss function
loss_function = mse

# accuracy metric
metric = torchmetrics.functional.accuracy

# display model
print(model.parameters)

<bound method Module.parameters of MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=27, out_features=18, bias=True)
    (1): Tanh()
    (2): Linear(in_features=18, out_features=12, bias=True)
    (3): Tanh()
    (4): Linear(in_features=12, out_features=8, bias=True)
    (5): Tanh()
    (6): Linear(in_features=8, out_features=1, bias=True)
    (7): Sigmoid()
  )
)>


#### Training and test

In [47]:
training_results = []
training_losses = []
testing_results = []

training_results, training_losses, testing_results = evaluate(training_s,
                                                            test_s,
                                                            model,
                                                            loss_function,
                                                            metric,
                                                            optimizer,
                                                            epochs,
                                                            early_stop=False)

Epoch 1 / 150
	Training	Accuracy: 0.4117647111	Loss: 0.2543039322
	Test error	Accuracy: 0.3972602785	Loss: 0.2505727914
Epoch 2 / 150
	Training	Accuracy: 0.4117647111	Loss: 0.2536374927
	Test error	Accuracy: 0.4109589159	Loss: 0.2499907620
Epoch 3 / 150
	Training	Accuracy: 0.4117647111	Loss: 0.2529648542
	Test error	Accuracy: 0.4246575236	Loss: 0.2494086623
Epoch 4 / 150
	Training	Accuracy: 0.4117647111	Loss: 0.2522921264
	Test error	Accuracy: 0.4246575236	Loss: 0.2488209307
Epoch 5 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2516234815
	Test error	Accuracy: 0.4246575236	Loss: 0.2482315983
Epoch 6 / 150
	Training	Accuracy: 0.4705882370	Loss: 0.2509469688
	Test error	Accuracy: 0.4657534361	Loss: 0.2476400946
Epoch 7 / 150
	Training	Accuracy: 0.5294117928	Loss: 0.2502631545
	Test error	Accuracy: 0.4794520438	Loss: 0.2470382360
Epoch 8 / 150
	Training	Accuracy: 0.5588235259	Loss: 0.2495728731
	Test error	Accuracy: 0.5068492889	Loss: 0.2464318602
Epoch 9 / 150
	Training	Accuracy: 0.5882

	Test error	Accuracy: 0.9041095972	Loss: 0.1880296611
Epoch 70 / 150
	Training	Accuracy: 0.8823529482	Loss: 0.1835405380
	Test error	Accuracy: 0.9041095972	Loss: 0.1869065882
Epoch 71 / 150
	Training	Accuracy: 0.8823529482	Loss: 0.1822725534
	Test error	Accuracy: 0.9041095972	Loss: 0.1857641701
Epoch 72 / 150
	Training	Accuracy: 0.8823529482	Loss: 0.1810634583
	Test error	Accuracy: 0.9041095972	Loss: 0.1846596279
Epoch 73 / 150
	Training	Accuracy: 0.8823529482	Loss: 0.1798195243
	Test error	Accuracy: 0.9041095972	Loss: 0.1835189951
Epoch 74 / 150
	Training	Accuracy: 0.8823529482	Loss: 0.1785971075
	Test error	Accuracy: 0.9041095972	Loss: 0.1824025817
Epoch 75 / 150
	Training	Accuracy: 0.8823529482	Loss: 0.1773921251
	Test error	Accuracy: 0.9041095972	Loss: 0.1812931264
Epoch 76 / 150
	Training	Accuracy: 0.8823529482	Loss: 0.1761306226
	Test error	Accuracy: 0.9041095972	Loss: 0.1801400951
Epoch 77 / 150
	Training	Accuracy: 0.8823529482	Loss: 0.1749187112
	Test error	Accuracy: 0.91780823

	Training	Accuracy: 0.9705882072	Loss: 0.1031968296
	Test error	Accuracy: 0.9726027250	Loss: 0.1115540592
Epoch 138 / 150
	Training	Accuracy: 0.9705882072	Loss: 0.1021637619
	Test error	Accuracy: 0.9726027250	Loss: 0.1105535591
Epoch 139 / 150
	Training	Accuracy: 0.9705882072	Loss: 0.1011267304
	Test error	Accuracy: 0.9726027250	Loss: 0.1095575378
Epoch 140 / 150
	Training	Accuracy: 0.9705882072	Loss: 0.1000872627
	Test error	Accuracy: 0.9726027250	Loss: 0.1085487076
Epoch 141 / 150
	Training	Accuracy: 0.9705882072	Loss: 0.0990645513
	Test error	Accuracy: 0.9726027250	Loss: 0.1075586304
Epoch 142 / 150
	Training	Accuracy: 0.9705882072	Loss: 0.0980521441
	Test error	Accuracy: 0.9726027250	Loss: 0.1065826916
Epoch 143 / 150
	Training	Accuracy: 0.9705882072	Loss: 0.0970551744
	Test error	Accuracy: 0.9726027250	Loss: 0.1056215284
Epoch 144 / 150
	Training	Accuracy: 0.9705882072	Loss: 0.0960655883
	Test error	Accuracy: 0.9726027250	Loss: 0.1046582780
Epoch 145 / 150
	Training	Accuracy: 0.97

#### Prepare tanh results for analysis

In [48]:
tailend = f"Tanh ({prefix})"

train_preds, test_preds = prep(tailend, training_results, testing_results)

#### Generate tensorboard summary

In [49]:
to_tensorboard(tailend, directory, training_s, train_preds, test_preds, y_train, y_test)

#### Store Tanh results in dataframe 

In [50]:
df["pred_tanh"] = np.array(np.round(test_preds[-1])).astype(int)
name = f"{prefix}_tanh_{suffix}.pth"
custom_save(name, model)
#torch.save(model, f"{prefix}_tanh_scaled.pth")

compas_tanh_mk2_FTU.pth has been saved.


In [51]:
df.head()

,age,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,is_recid,is_violent_recid,decile_score.1,v_decile_score,priors_count.1,start,end,sex,age_cat_25to45,age_cat_over45,age_cat_under25,race_asian,c_charge_degree,score_text_High,score_text_Low,score_text_Medium,v_score_text_High,v_score_text_Low,v_score_text_Medium,event,black,white,hispanic,native,other,target,pred_relu,pred_tanh
0,25,0,5,0,1,4,-1,1,1,0,5,4,4,32,304,0,1,0,0,0,1,0,0,1,0,1,0,1,1,0,0,0,0,1,1,1
1,35,0,4,0,0,3,-1,0,1,0,4,4,3,9,462,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,1,1
2,32,0,4,0,0,1,0,0,0,0,4,3,1,0,1171,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0
3,20,0,10,0,0,0,-1,1,1,1,10,10,0,3,20,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,0,1,1,1
4,50,0,8,0,2,24,0,1,1,1,8,8,24,32,87,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,1


## Trial 3.3 - ELU (COMPAS)<a id="elu"></a>

#### Hyperparameters

In [52]:
#alpha = 1e-5 # 0.00001
#epochs = 50
#batches = 256
hidden1 = elu
hidden2 = elu
hidden3 = elu
out_layer = sigmoid
print(f"learning rate: {alpha}")
print(f"epochs: {epochs}")
print(f"batches: {batches}")

learning rate: 1e-05
epochs: 150
batches: 100


#### Construct model

In [53]:
# instantiate model
model = MLP(X_train.shape[1], hidden1, hidden2, hidden3, out_layer).to(device)

# select optimizing function
optimizer = adam(model.parameters(), lr=alpha)

# select loss function
criterion = mse

# accuracy metric
metric = torchmetrics.functional.accuracy

# display model
print(model.parameters)

<bound method Module.parameters of MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=27, out_features=18, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=18, out_features=12, bias=True)
    (3): ELU(alpha=1.0)
    (4): Linear(in_features=12, out_features=8, bias=True)
    (5): ELU(alpha=1.0)
    (6): Linear(in_features=8, out_features=1, bias=True)
    (7): Sigmoid()
  )
)>


#### Train and test

In [54]:
training_results = []
training_losses = []
testing_results = []

training_results, training_losses, testing_results = evaluate(training_s,
                                                            test_s,
                                                            model,
                                                            loss_function,
                                                            metric,
                                                            optimizer,
                                                            epochs,
                                                            early_stop=False)

Epoch 1 / 150
	Training	Accuracy: 0.5588235259	Loss: 0.2476783395
	Test error	Accuracy: 0.6027397513	Loss: 0.2511324812
Epoch 2 / 150
	Training	Accuracy: 0.5588235259	Loss: 0.2472200990
	Test error	Accuracy: 0.6027397513	Loss: 0.2507132285
Epoch 3 / 150
	Training	Accuracy: 0.5588235259	Loss: 0.2467676550
	Test error	Accuracy: 0.6027397513	Loss: 0.2502936536
Epoch 4 / 150
	Training	Accuracy: 0.5588235259	Loss: 0.2463148981
	Test error	Accuracy: 0.6027397513	Loss: 0.2498707835
Epoch 5 / 150
	Training	Accuracy: 0.5588235259	Loss: 0.2458629012
	Test error	Accuracy: 0.6027397513	Loss: 0.2494533019
Epoch 6 / 150
	Training	Accuracy: 0.5588235259	Loss: 0.2454108149
	Test error	Accuracy: 0.6027397513	Loss: 0.2490300742
Epoch 7 / 150
	Training	Accuracy: 0.5588235259	Loss: 0.2449523211
	Test error	Accuracy: 0.6027397513	Loss: 0.2486070387
Epoch 8 / 150
	Training	Accuracy: 0.5588235259	Loss: 0.2444974929
	Test error	Accuracy: 0.6027397513	Loss: 0.2481863279
Epoch 9 / 150
	Training	Accuracy: 0.5588

	Training	Accuracy: 0.7941176295	Loss: 0.1956461817
	Test error	Accuracy: 0.8493150473	Loss: 0.2031952725
Epoch 71 / 150
	Training	Accuracy: 0.7941176295	Loss: 0.1945047379
	Test error	Accuracy: 0.8493150473	Loss: 0.2021525509
Epoch 72 / 150
	Training	Accuracy: 0.7941176295	Loss: 0.1933717877
	Test error	Accuracy: 0.8493150473	Loss: 0.2011040776
Epoch 73 / 150
	Training	Accuracy: 0.7941176295	Loss: 0.1922188997
	Test error	Accuracy: 0.8493150473	Loss: 0.2000398792
Epoch 74 / 150
	Training	Accuracy: 0.7941176295	Loss: 0.1910829246
	Test error	Accuracy: 0.8493150473	Loss: 0.1989885122
Epoch 75 / 150
	Training	Accuracy: 0.7941176295	Loss: 0.1899308264
	Test error	Accuracy: 0.8356164098	Loss: 0.1979228294
Epoch 76 / 150
	Training	Accuracy: 0.7647058964	Loss: 0.1887672693
	Test error	Accuracy: 0.8356164098	Loss: 0.1968534028
Epoch 77 / 150
	Training	Accuracy: 0.7647058964	Loss: 0.1876121759
	Test error	Accuracy: 0.8356164098	Loss: 0.1957783713
Epoch 78 / 150
	Training	Accuracy: 0.7647058964

	Training	Accuracy: 0.9117646813	Loss: 0.1129222512
	Test error	Accuracy: 0.9452054501	Loss: 0.1237063834
Epoch 140 / 150
	Training	Accuracy: 0.9117646813	Loss: 0.1117158905
	Test error	Accuracy: 0.9452054501	Loss: 0.1224985928
Epoch 141 / 150
	Training	Accuracy: 0.9117646813	Loss: 0.1105018482
	Test error	Accuracy: 0.9589040875	Loss: 0.1212722922
Epoch 142 / 150
	Training	Accuracy: 0.9117646813	Loss: 0.1093001664
	Test error	Accuracy: 0.9589040875	Loss: 0.1200640265
Epoch 143 / 150
	Training	Accuracy: 0.9117646813	Loss: 0.1080906764
	Test error	Accuracy: 0.9589040875	Loss: 0.1188549236
Epoch 144 / 150
	Training	Accuracy: 0.9117646813	Loss: 0.1068835407
	Test error	Accuracy: 0.9589040875	Loss: 0.1176474215
Epoch 145 / 150
	Training	Accuracy: 0.9117646813	Loss: 0.1056938469
	Test error	Accuracy: 0.9589040875	Loss: 0.1164503310
Epoch 146 / 150
	Training	Accuracy: 0.9117646813	Loss: 0.1044935882
	Test error	Accuracy: 0.9589040875	Loss: 0.1152470640
Epoch 147 / 150
	Training	Accuracy: 0.91

#### Prepare ELU results for analysis

In [55]:
tailend = f"ELU ({prefix})"

train_preds, test_preds = prep(tailend, training_results, testing_results)

#### Save metrics to tensorboad

In [56]:
to_tensorboard(tailend, directory, training_s, train_preds, test_preds, y_train, y_test)

#### Add ELU test results to persistent storage

In [57]:
df["pred_elu"] = np.array(np.round(test_preds[-1])).astype(int)
name = f"{prefix}_elu_{suffix}.pth"
custom_save(name, model, 1)
#torch.save(model, f"{prefix}_elu_scaled.pth")

compas_elu_mk2_FTU.pth has been saved.


In [58]:
df.head()

,age,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,is_recid,is_violent_recid,decile_score.1,v_decile_score,priors_count.1,start,end,sex,age_cat_25to45,age_cat_over45,age_cat_under25,race_asian,c_charge_degree,score_text_High,score_text_Low,score_text_Medium,v_score_text_High,v_score_text_Low,v_score_text_Medium,event,black,white,hispanic,native,other,target,pred_relu,pred_tanh,pred_elu
0,25,0,5,0,1,4,-1,1,1,0,5,4,4,32,304,0,1,0,0,0,1,0,0,1,0,1,0,1,1,0,0,0,0,1,1,1,1
1,35,0,4,0,0,3,-1,0,1,0,4,4,3,9,462,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,1,1,1
2,32,0,4,0,0,1,0,0,0,0,4,3,1,0,1171,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0
3,20,0,10,0,0,0,-1,1,1,1,10,10,0,3,20,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,0,1,1,1,1
4,50,0,8,0,2,24,0,1,1,1,8,8,24,32,87,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,1,1


## Trial 3.4 - Leaky ReLU (COMPAS) <a id="leaky"></a>

#### Hyperparameters

In [59]:
#alpha = 1e-5 # 0.00001
#epochs = 50
#batches = 256
hidden1 = leaky
hidden2 = leaky
hidden3 = leaky
out_layer = sigmoid
print(f"learning rate: {alpha}")
print(f"epochs: {epochs}")
print(f"batches: {batches}")

learning rate: 1e-05
epochs: 150
batches: 100


#### Construct Model

In [60]:
# instantiate model
model = MLP(X_train.shape[1], hidden1, hidden2, hidden3, out_layer).to(device)

# select optimizing function
optimizer = adam(model.parameters(), lr=alpha)

# select loss function
criterion = mse

# accuracy metric
metric = torchmetrics.functional.accuracy

# display model
print(model.parameters)

<bound method Module.parameters of MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=27, out_features=18, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=18, out_features=12, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=12, out_features=8, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=8, out_features=1, bias=True)
    (7): Sigmoid()
  )
)>


#### Train and test

In [61]:
training_results = []
training_losses = []
testing_results = []

training_results, training_losses, testing_results = evaluate(training_s,
                                                             test_s,
                                                             model,
                                                             loss_function,
                                                             metric,
                                                             optimizer,
                                                             epochs,
                                                             early_stop=False)

Epoch 1 / 150
	Training	Accuracy: 0.6764705777	Loss: 0.2483161539
	Test error	Accuracy: 0.6164383292	Loss: 0.2481228773
Epoch 2 / 150
	Training	Accuracy: 0.7058823705	Loss: 0.2480029464
	Test error	Accuracy: 0.6301369667	Loss: 0.2478469199
Epoch 3 / 150
	Training	Accuracy: 0.7058823705	Loss: 0.2476756424
	Test error	Accuracy: 0.6438356042	Loss: 0.2475536728
Epoch 4 / 150
	Training	Accuracy: 0.7058823705	Loss: 0.2473300993
	Test error	Accuracy: 0.6438356042	Loss: 0.2472428864
Epoch 5 / 150
	Training	Accuracy: 0.7647058964	Loss: 0.2469552904
	Test error	Accuracy: 0.6575342417	Loss: 0.2469156789
Epoch 6 / 150
	Training	Accuracy: 0.7647058964	Loss: 0.2465411425
	Test error	Accuracy: 0.6849315166	Loss: 0.2465683257
Epoch 7 / 150
	Training	Accuracy: 0.7352941036	Loss: 0.2461168617
	Test error	Accuracy: 0.7260273695	Loss: 0.2462090644
Epoch 8 / 150
	Training	Accuracy: 0.7352941036	Loss: 0.2456738204
	Test error	Accuracy: 0.7534246445	Loss: 0.2458360756
Epoch 9 / 150
	Training	Accuracy: 0.7352

	Training	Accuracy: 0.8235294223	Loss: 0.2046903521
	Test error	Accuracy: 0.8493150473	Loss: 0.2066729885
Epoch 71 / 150
	Training	Accuracy: 0.8235294223	Loss: 0.2038003653
	Test error	Accuracy: 0.8493150473	Loss: 0.2058287469
Epoch 72 / 150
	Training	Accuracy: 0.8235294223	Loss: 0.2029171884
	Test error	Accuracy: 0.8493150473	Loss: 0.2049824255
Epoch 73 / 150
	Training	Accuracy: 0.8235294223	Loss: 0.2020072043
	Test error	Accuracy: 0.8493150473	Loss: 0.2041255967
Epoch 74 / 150
	Training	Accuracy: 0.8235294223	Loss: 0.2010989189
	Test error	Accuracy: 0.8493150473	Loss: 0.2032664177
Epoch 75 / 150
	Training	Accuracy: 0.8235294223	Loss: 0.2001856714
	Test error	Accuracy: 0.8493150473	Loss: 0.2024043401
Epoch 76 / 150
	Training	Accuracy: 0.8529411554	Loss: 0.1992606223
	Test error	Accuracy: 0.8493150473	Loss: 0.2015349837
Epoch 77 / 150
	Training	Accuracy: 0.8529411554	Loss: 0.1983191222
	Test error	Accuracy: 0.8493150473	Loss: 0.2006485654
Epoch 78 / 150
	Training	Accuracy: 0.8823529482

	Training	Accuracy: 0.9411764741	Loss: 0.1341151297
	Test error	Accuracy: 0.9315068722	Loss: 0.1392619471
Epoch 139 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1330289096
	Test error	Accuracy: 0.9315068722	Loss: 0.1382295833
Epoch 140 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1319509596
	Test error	Accuracy: 0.9315068722	Loss: 0.1371893095
Epoch 141 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1308664382
	Test error	Accuracy: 0.9315068722	Loss: 0.1361500326
Epoch 142 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1297907531
	Test error	Accuracy: 0.9315068722	Loss: 0.1351150118
Epoch 143 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1287105381
	Test error	Accuracy: 0.9315068722	Loss: 0.1340609051
Epoch 144 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1276101321
	Test error	Accuracy: 0.9315068722	Loss: 0.1330216992
Epoch 145 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1265446991
	Test error	Accuracy: 0.9315068722	Loss: 0.1319839873
Epoch 146 / 150
	Training	Accuracy: 0.94

#### Prepare Leaky ReLU data for analysis

In [62]:
tailend = f"Leaky ReLU ({prefix})"

train_preds, test_preds = prep(tailend, training_results, testing_results)

#### Add Leaky ReLU data to persistent storage

In [63]:
df["pred_leaky"] = np.array(np.round(test_preds[-1])).astype(int)
name = f"{prefix}_leaky_{suffix}.pth"
custom_save(name, model,1)
# torch.save(model, "loans_leaky_scaled.pth")

compas_leaky_mk2_FTU.pth has been saved.


In [64]:
df.head()

,age,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,is_recid,is_violent_recid,decile_score.1,v_decile_score,priors_count.1,start,end,sex,age_cat_25to45,age_cat_over45,age_cat_under25,race_asian,c_charge_degree,score_text_High,score_text_Low,score_text_Medium,v_score_text_High,v_score_text_Low,v_score_text_Medium,event,black,white,hispanic,native,other,target,pred_relu,pred_tanh,pred_elu,pred_leaky
0,25,0,5,0,1,4,-1,1,1,0,5,4,4,32,304,0,1,0,0,0,1,0,0,1,0,1,0,1,1,0,0,0,0,1,1,1,1,1
1,35,0,4,0,0,3,-1,0,1,0,4,4,3,9,462,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,1,1,1,1
2,32,0,4,0,0,1,0,0,0,0,4,3,1,0,1171,0,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0
3,20,0,10,0,0,0,-1,1,1,1,10,10,0,3,20,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,0,1,1,1,1,1
4,50,0,8,0,2,24,0,1,1,1,8,8,24,32,87,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,1,1,1,1


# Results

## Save data

In [65]:
name = f"{prefix}_results_{suffix}.parquet"
custom_save(name, df, 0)

compas_results_mk2_FTU.parquet has been saved.
